### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 移除671個詞沒有完全對應的資料點
- 以句點為分隔拆句子
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np
import pynlpir
from sklearn.metrics import classification_report

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
pynlpir.open()  # 打开分词器

filename = "dictionary.txt"
with open(filename) as f:
    lines = f.readlines()

dic_terms = []
stopwords = [' ','.','。','，','、','(',')','為','又','等','或']
for line in lines:
    line = line.strip()
    line = line.replace('.','')
    seg_list = pynlpir.segment(line)
    for t in seg_list:
        if t[0].isdigit() or t[0] in stopwords:
            continue
        dic_terms.append(t[0])

['自觉', '症状', '一', '寒', '热', '汗', '症状', '一', '恶', '寒', '类', '恶', '寒', '怕', '冷', '取暖', '不得', '解', '即', '病人', '时时', '觉', '冷', '虽', '加', '衣', '履', '被', '近', '火', '取', '暖', '犹', '不', '能', '解', '其', '寒', '的', '症状', '见于', '外感', '风寒', '表', '实证', '微', '恶', '寒', '恶', '寒', '的', '程度', '较', '轻', '可见', '于', '多种', '表', '证', '如', '风寒', '初', '起', '新', '感', '风', '热', '证', '称', '微', '恶', '风寒', '背', '恶', '寒', '仅', '感觉', '背部', '背脊', '局部', '恶', '寒', '见于', '风寒', '羁', '于', '太阳', '经脉', '内伤', '脾胃', '虚', '寒', '寒', '饮', '停', '聚', '证', '寒战', '恶', '寒', '的', '同时', '伴有', '肢体', '振动', '战', '栗', '即', '表现', '为', '怕', '冷', '且', '全身', '不', '自', '主', '地', '颤动', '的', '症状', '见于', '邪', '正', '交', '争', '激烈', '外感', '风寒', '重', '证', '疫疠', '表', '证', '阳', '虚', '寒', '凝', '证', '热', '毒', '内', '陷', '证', '疟疾', '疔', '疮', '走', '黄', '疽', '毒', '内', '陷', '病', '证', '称', '为', '恶', '寒战', '栗', '寒', '栗', '战', '寒', '振', '寒', '恶', '风', '病人', '自觉', '怕', '风', '遇', '风', '则', '怕', '冷战', '栗', '避风', '则', '缓解', '是', '风', '邪', '袭', '表', '卫', '气', '失和', '的', 

In [5]:
pynlpir.open()  # 打开分词器

filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    #原文編碼有問題，跳過
    if idx in [64,223,251,389,480,673,744,751,769,1422,1541,1853]:
            continue
    
    #症狀詞作斷詞
    ne_list = []
    for term in main_list:
        terms_list = pynlpir.segment(term)
        for t in terms_list:
            ne_list.append(t[0])
    
    #每個醫案拆句
    for sent in sentences:
        
        word_list = []
    
        txt_list = pynlpir.segment(sent)
        for txt in txt_list:
            if txt[0] in ne_list:
                e = (txt[0],txt[1],"S")
            else :
                e = (txt[0],txt[1],"O")
            word_list.append(e)

        docs.append(word_list)
    
print(len(docs))

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: '

48356


In [ ]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 48901
print(ave)

In [6]:
data = docs

In [8]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    indic = False
    
    if postag is None:
        postag = 'none'
            
    if word in dic_terms:
        indic = True
    else:
        indic = False

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'indic=%s' % indic
    ]
        
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        if word1 in dic_terms:
            indic = True
        else:
            indic = False
            
        features.extend([
            '-1:word=' + word1,
            '-1.word.isdigit=%s' % word.isdigit(),
            '-1:postag=' + postag1,
            'indic=%s' % indic
            
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        if word1 in dic_terms:
            indic = True
        else:
            indic = False
            
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1,
            'indic=%s' % indic
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [9]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
print(len(X_train))
print(len(X_test))

In [10]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 60315
Seconds required: 1.896

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 321828.334209
Feature norm: 1.000000
Error norm: 289262.990386
Active features: 59239
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 0.510

***** Iteration #2 *****
Loss: 283908.867409
Feature norm: 0.871253
Error norm: 255522.803714
Active features: 58085
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.235

***** Iteration #3 *****
Loss: 245203.163521
Feature norm: 0.536464
Error norm: 333005.328714
Active features: 37280
Line search trials: 2
Line search step: 0.500000
Seconds requir

***** Iteration #45 *****
Loss: 102011.068335
Feature norm: 71.890885
Error norm: 11761.753038
Active features: 29089
Line search trials: 6
Line search step: 0.031250
Seconds required for this iteration: 1.457

***** Iteration #46 *****
Loss: 101205.035784
Feature norm: 76.014207
Error norm: 3995.661862
Active features: 28907
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.248

***** Iteration #47 *****
Loss: 100188.556293
Feature norm: 83.675690
Error norm: 10913.195512
Active features: 28641
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.241

***** Iteration #48 *****
Loss: 99317.099174
Feature norm: 90.852676
Error norm: 8636.991009
Active features: 28571
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.247

***** Iteration #49 *****
Loss: 98526.115095
Feature norm: 98.916885
Error norm: 7427.471494
Active features: 28706
Line search trials: 1
Line search step: 1.0

***** Iteration #91 *****
Loss: 91462.341013
Feature norm: 199.746239
Error norm: 1673.422819
Active features: 25570
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.246

***** Iteration #92 *****
Loss: 91447.592058
Feature norm: 199.884931
Error norm: 3144.008646
Active features: 25475
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.244

***** Iteration #93 *****
Loss: 91429.730051
Feature norm: 200.060323
Error norm: 2715.434455
Active features: 25410
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.241

***** Iteration #94 *****
Loss: 91414.954098
Feature norm: 200.148758
Error norm: 2200.972899
Active features: 25334
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.246

***** Iteration #95 *****
Loss: 91400.072325
Feature norm: 200.241780
Error norm: 1277.499438
Active features: 25321
Line search trials: 1
Line search step: 1.0

***** Iteration #134 *****
Loss: 91106.999252
Feature norm: 203.238098
Error norm: 1859.990555
Active features: 24323
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.246

***** Iteration #135 *****
Loss: 91103.141717
Feature norm: 203.271128
Error norm: 1944.638954
Active features: 24311
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.247

***** Iteration #136 *****
Loss: 91099.392672
Feature norm: 203.314506
Error norm: 2002.275407
Active features: 24289
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.243

***** Iteration #137 *****
Loss: 91095.412911
Feature norm: 203.349711
Error norm: 1695.914005
Active features: 24279
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.241

***** Iteration #138 *****
Loss: 91091.696325
Feature norm: 203.396471
Error norm: 1673.643653
Active features: 24277
Line search trials: 1
Line search step

***** Iteration #174 *****
Loss: 90979.310738
Feature norm: 204.633021
Error norm: 1071.463630
Active features: 23733
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.250

***** Iteration #175 *****
Loss: 90977.119790
Feature norm: 204.647380
Error norm: 1429.927296
Active features: 23735
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.247

***** Iteration #176 *****
Loss: 90974.686712
Feature norm: 204.688544
Error norm: 1087.325642
Active features: 23725
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.244

***** Iteration #177 *****
Loss: 90972.567814
Feature norm: 204.707968
Error norm: 1241.741559
Active features: 23716
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.248

***** Iteration #178 *****
Loss: 90970.191686
Feature norm: 204.738092
Error norm: 771.903485
Active features: 23705
Line search trials: 1
Line search step:

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [12]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "S"]))

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "S"]))

             precision    recall  f1-score   support

          O       0.97      0.98      0.97    856008
          S       0.71      0.57      0.64     68383

avg / total       0.95      0.95      0.95    924391

             precision    recall  f1-score   support

          O       0.96      0.98      0.97    211510
          S       0.67      0.53      0.59     17143

avg / total       0.94      0.95      0.94    228653

